## Python notebook for generating the STAC catalog json and corresponding Item json for Raster & Vector layers

### Tools:
1. Pystac
2. Rasterio
3. Geopandas
4. Matplotlib

This notebook returns Catalog json for Raster and Vector layers.

### 1. Importing the required modules

In [88]:
import os
import json
import xml.etree.ElementTree as ET
from datetime import datetime
from glob import glob

import rasterio
from rasterio.warp import transform_bounds
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
import pystac
import sys
import numpy as np
from shapely.geometry import mapping, box
from pystac.extensions.table import TableExtension
from pystac import Asset, MediaType
from pystac.extensions.classification import ClassificationExtension, Classification
from pystac.extensions.raster import RasterExtension,RasterBand
from pystac.extensions.projection import ProjectionExtension
import constants

### 2. Defining the variables used in the notebook

In [89]:
base_dir="data/"

blocks_info = [
    {
        "block": "gobindpur",
        "location": "jharkhand",
        "raster_file": "saraikela-kharsawan_gobindpur_2023-07-01_2024-06-30_LULCmap_10m.tif",
        "vector_file": "swb2_saraikela-kharsawan_gobindpur.geojson",
        "raster_style_file": "style_file.qml",
        "vector_style_file": "swb_style.qml"
    },
    {
        "block": "mirzapur",
        "location": "uttar_pradesh",
        "raster_file":"Mirzapur_Mirzapur_2023-07-01_2024-06-30_LULCmap_10m.tif",
        "vector_file":"surface_waterbodies_mirzapur_mirzapur.geojson",
        "raster_style_file": "style_file.qml",
        "vector_style_file": "swb_style.qml"
    },
    {
        "block": "koraput",
        "location": "odisha",
        "raster_file": "Narayanpatana_Koraput_2023-07-01_2024-06-30_LULCmap_10m.tif",
        "vector_file": "surface_waterbodies_koraput_narayanpatana.geojson",
        "raster_style_file": "style_file.qml",
        "vector_style_file": "swb_style.qml"
    },
    {
        "block": "Badlapur",
        "location": "uttar_pradesh",
        "admin_boundary_file":"admin_boundary_jaunpur_badlapur.geojson",
        "nrega_assets_file":"jaunpur_badlapur.geojson",
        "lulc_raster_file":"jaunpur_badlapur_2019-07-01_2020-06-30_LULCmap_10m.tif",
        "admin_boundary_style_file":"Administrative-Boundary-Style.qml",
        "nrega_assets_style_file":"swb_style.qml",
        "lulc_raster_style_file":"level-3.qml"
    }
]

corestack_dir = os.path.join(base_dir, 'CorestackCatalogs')

### 3. For Raster layers the data range fecthed from filename

In [90]:
def extract_raster_dates_from_filename(raster_filename):
    try:
        print(raster_filename)
        parts = raster_filename.split('_')
        start_date = datetime.strptime(parts[2], "%Y-%m-%d")
        end_date = datetime.strptime(parts[3], "%Y-%m-%d")
        print(start_date)
        print(end_date)
    except Exception as e:
        raise ValueError(f"Failed to extract raster dates from filename '{raster_filename}': {e}")
        
    return start_date, end_date

### 4. Parsing the QML file for Raster Layers

In [91]:
def parse_qml_classes(qml_path):
    tree = ET.parse(qml_path)
    root = tree.getroot()
    classes = []

    for entry in root.findall(".//paletteEntry"):
        class_info = {}
        for attr_key, attr_value in entry.attrib.items():
            if attr_key == "value":
                try:
                    class_info[attr_key] = int(attr_value)
                except ValueError:
                    class_info[attr_key] = attr_value
            else:
                class_info[attr_key] = attr_value
        classes.append(class_info)
    return classes

### 5. Generating the thumbnails from the files 

In [92]:
def generate_vector_thumbnail(vector_path, out_path):
    gdf = gpd.read_file(vector_path)
    if gdf.crs is None or gdf.crs.to_epsg() != 4326:
        gdf = gdf.to_crs(epsg=4326)
    fig, ax = plt.subplots(figsize=(3, 3))
    fig.patch.set_facecolor("white")
    ax.set_facecolor("white")
    gdf.plot(ax=ax, color="lightblue", edgecolor="blue", linewidth=0.5)
    ax.axis('off')
    plt.savefig(out_path, dpi=150, bbox_inches='tight', pad_inches=0, facecolor=fig.get_facecolor())
    plt.close()

In [93]:
def generate_raster_thumbnail(tif_path, out_path, qml_path):
    with rasterio.open(tif_path) as src:
        arr = src.read(1) 
        nodata = src.nodata
        if nodata is not None:
            arr = np.ma.masked_equal(arr, nodata)
    
    unique_raster_values = np.unique(arr.compressed() if isinstance(arr, np.ma.MaskedArray) else arr)
    print(f"Unique values in raster data: {unique_raster_values}")
    
    style_info = parse_qml_classes(qml_path)

    # Filter QML info to only include values present in the raster data
    filtered_style_info = [cls for cls in style_info if cls.get('value') in unique_raster_values]
    
    values = [cls['value'] for cls in filtered_style_info if 'value' in cls]
    colors = [cls['color'] for cls in filtered_style_info if 'color' in cls]
    
    print(f"Parsed QML values: {values}")
    print(f"Parsed QML colors: {colors}")
    
    if not values or not colors or len(values) != len(colors):
        raise ValueError("Invalid or insufficient palette information in QML file.")

    sorted_indices = np.argsort(values)
    sorted_values = np.array(values)[sorted_indices]
    sorted_colors = np.array(colors)[sorted_indices]

    cmap = ListedColormap(sorted_colors)

    bounds = np.array(sorted_values) - 0.5
    bounds = np.append(bounds, sorted_values[-1] + 0.5)

    norm = Normalize(vmin=bounds.min(), vmax=bounds.max())

    plt.figure(figsize=(3, 3), dpi=100)
    
    plt.imshow(arr, cmap=cmap, norm=norm, interpolation='none')
    plt.axis('off')

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    plt.savefig(out_path, bbox_inches='tight', pad_inches=0)
    plt.close()

### 6. Creating the Raster items and adding the assets

In [ ]:
def create_raster_item(location, block, raster_filename, raster_path, raster_dir, raster_thumbnail, raster_style_file, base_dir):
    try:
        start_date, end_date = extract_raster_dates_from_filename(raster_filename=raster_filename)
    except ValueError as e:
        raise RuntimeError(f"Raster item creation failed")
    
    with rasterio.open(raster_path) as src:
        bounds = src.bounds
        data_type = str(src.dtypes[0])
        nodata = src.nodata if src.nodata is not None else 0
        height, width = src.shape
        transform = src.transform
        
        proj_epsg = None
        if src.crs and src.crs.is_epsg_code:
            proj_epsg = src.crs.to_epsg()
        
        if proj_epsg != 32644:
            reprojected_bounds = transform_bounds(src.crs, 'EPSG:32644', *bounds)
            bbox = list(reprojected_bounds)
            wgs84_bbox = bbox
            gsd_x = (reprojected_bounds[2] - reprojected_bounds[0]) / width
            gsd_y = (reprojected_bounds[3] - reprojected_bounds[1]) / height
            gsd = (gsd_x + gsd_y) / 2
        else:
            bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
            wgs84_bbox = transform_bounds(src.crs, 'epsg:4326', bounds.left, bounds.bottom, bounds.right, bounds.top)
            gsd = src.res[0]
            
    print(f"Raster resolution (GSD): {gsd} meters")
    geom = mapping(box(*wgs84_bbox))

    generate_raster_thumbnail(raster_path, raster_thumbnail, raster_style_file)
    style_info = parse_qml_classes(raster_style_file)

    style_json_path = os.path.join(raster_dir, os.path.basename(raster_style_file).replace('.qml', '.json'))
    with open(style_json_path, "w") as f:
        json.dump(style_info, f, indent=2)

    item = pystac.Item(
        id=os.path.splitext(raster_filename)[0],
        geometry=geom,
        bbox=wgs84_bbox,
        datetime=start_date,
        properties={
            "title": "LULC Raster Layer",
            "description": f"Raster data for {os.path.splitext(raster_filename)[0]} in {block} of {location}",
            "start_datetime": start_date.isoformat() + 'Z',
            "end_datetime": end_date.isoformat() + 'Z',
            "gsd": "gsd",
        }
    )

    # Add Projection extension properties to the ITEM
    proj_ext = ProjectionExtension.ext(item, add_if_missing=True)
    proj_ext.epsg = proj_epsg
    proj_ext.bbox = bbox
    proj_ext.shape = [src.height, src.width]
    proj_ext.transform = list(transform)
        
    item.add_asset("data", Asset(
        href=os.path.relpath(raster_path, start=os.path.dirname(raster_dir)),
        media_type=MediaType.GEOTIFF,
        roles=["data"],
        title="Raster Layer"
    ))

    raster_ext = RasterExtension.ext(item.assets["data"], add_if_missing=True)
    raster_band = RasterBand.create(
        data_type=data_type, 
        spatial_resolution=gsd,
        nodata=nodata
    )
    raster_ext.bands = [raster_band]

    classification_ext = ClassificationExtension.ext(item.assets["data"], add_if_missing=True)
    stac_classes = []
    for cls in style_info:
        stac_class_obj = Classification.create(
            value=int(cls["value"]),
            name=cls.get("label") or f"Class {cls['value']}",
            description=cls.get("label"),
            color_hint=cls['color'].replace('#','')
        )
        stac_classes.append(stac_class_obj)
    classification_ext.classes = stac_classes

    item.add_asset("thumbnail", Asset(
        href=os.path.relpath(raster_thumbnail, start=os.path.dirname(raster_dir)),
        media_type=MediaType.PNG,
        roles=["thumbnail"],
        title="Raster Thumbnail"
    ))

    item.add_asset("legend", Asset(
        href=os.path.relpath(style_json_path, start=os.path.dirname(raster_dir)),
        media_type=MediaType.JSON,
        roles=["metadata"],
        title="Legend JSON"
    ))

    item.add_asset("style", Asset(
        href=os.path.relpath(raster_style_file, start=os.path.dirname(raster_dir)),
        media_type=MediaType.XML,
        roles=["metadata"],
        title="Raster Style (QML)"
    ))

    item.set_self_href(os.path.join(raster_dir, f"{os.path.splitext(raster_filename)[0]}.json"))
    item.save_object()
    return item

### 7.Creating the Vector items and adding the assets

In [95]:
def parse_vector_descriptions(qml_path):
    tree = ET.parse(qml_path)
    root = tree.getroot()
    columns = []
    colnames = []
    coldesc = []
    for entry in root.findall(".//alias"):
        for attr_key, attr_value in entry.attrib.items():
            if (attr_key == 'field'):
                colnames.append(attr_value)
            if (attr_key == 'name'):
                coldesc.append(attr_value)
    vector_desc_df = pd.DataFrame([colnames,coldesc]).T
    vector_desc_df.columns = ['column_name','column_description']
    return vector_desc_df

In [96]:
def create_vector_item(location, block, vector_filename, vector_path, vector_desc_df, vector_dir, vector_thumbnail, vector_style_file, base_dir):
    start_date = constants.DEFAULT_START_DATE
    end_date = constants.DEFAULT_END_DATE

    if vector_filename.endswith('.geojson'):
        media_type = MediaType.GEOJSON
    
    gdf = gpd.read_file(vector_path)
    gdf_wgs84 = gdf.to_crs(epsg=4326) if gdf.crs is None or gdf.crs.to_epsg() != 4326 else gdf

    bounds = gdf_wgs84.total_bounds
    bbox = [float(b) for b in bounds]
    geom = mapping(gdf_wgs84.unary_union)
    
    generate_vector_thumbnail(vector_path, vector_thumbnail)

    item_id = os.path.splitext(vector_filename)[0]

    item = pystac.Item(
        id=item_id,
        geometry=geom,
        bbox=bbox,
        datetime=start_date,
        properties={
            "title": "Vector Layer",
            "description": f"Vector data for {os.path.splitext(vector_filename)[0]} in {block} of {location}",
            "start_datetime": start_date.isoformat() + 'Z',
            "end_datetime": end_date.isoformat() + 'Z',
        }
    )

    table_ext = TableExtension.ext(item, add_if_missing=True)

    vector_merged_df = gdf.dtypes.reset_index()
    vector_merged_df.columns = ['column_name','column_dtype']
    vector_merged_df = vector_merged_df.merge(vector_desc_df, on='column_name', how='left').fillna('')

    table_ext.columns = [
        {
            "name": row['column_name'],
            "type": str(row['column_dtype']),
            "description" : row['column_description']
        }
        for ind,row in vector_merged_df.iterrows()
    ]
    
    item.add_asset("data", Asset(
        href=os.path.relpath(vector_path, start=os.path.dirname(vector_dir)),
        media_type=media_type,
        roles=["data"],
        title="Vector Layer"
    ))

    item.add_asset("thumbnail", Asset(
        href=os.path.relpath(vector_thumbnail, start=os.path.dirname(vector_dir)),
        media_type=MediaType.PNG,
        roles=["thumbnail"],
        title="Vector Thumbnail"
    ))

    item.add_asset("style", Asset(
        href=os.path.relpath(vector_style_file, start=os.path.dirname(vector_dir)),
        media_type=MediaType.XML,
        roles=["metadata"],
        title="Vector Style"
    ))
    
    item.set_self_href(os.path.join(vector_dir, f"{os.path.splitext(vector_filename)[0]}.json"))
    item.save_object()

    return item

### 8. Generating the STAC catalog from the items 

In [97]:
def generate_stac_for_block(info):
    base_dir = 'data/'
    corestack_dir = os.path.join(base_dir, 'CorestackCatalogs')

    location = info['location']
    block = info['block']
    
    location_dir = os.path.join(corestack_dir, location)
    block_dir=os.path.join(location_dir, block)

    os.makedirs(block_dir, exist_ok=True)
    
    block_catalog = pystac.Catalog(
        id=block,
        title=f"STAC for {block}",
        description=f"STAC catalog for {block} block data in {location}"
    )

    layers_to_process = []
    if 'lulc_raster_file' in info:
        layers_to_process.append({'type': 'raster', 'file_key': 'lulc_raster_file', 'style_key': 'lulc_raster_style_file', 'dir_name': 'lulc_raster'})
    if 'raster_file' in info:
        layers_to_process.append({'type': 'raster', 'file_key': 'raster_file', 'style_key': 'raster_style_file', 'dir_name': 'raster'})
    if 'admin_boundary_file' in info:
        layers_to_process.append({'type': 'vector', 'file_key': 'admin_boundary_file', 'style_key': 'admin_boundary_style_file', 'dir_name': 'admin_boundary'})
    if 'nrega_assets_file' in info:
        layers_to_process.append({'type': 'vector', 'file_key': 'nrega_assets_file', 'style_key': 'nrega_assets_style_file', 'dir_name': 'nrega_assets'})
    if 'vector_file' in info:
        layers_to_process.append({'type': 'vector', 'file_key': 'vector_file', 'style_key': 'vector_style_file', 'dir_name': 'vector'})

    for layer in layers_to_process:
        file_path = os.path.join(base_dir, info[layer['file_key']])
        style_path = os.path.join(base_dir, info[layer['style_key']])
        
        if os.path.exists(file_path):
            layer_dir = os.path.join(block_dir, layer['dir_name'])
            os.makedirs(layer_dir, exist_ok=True)
            
            thumbnail_filename = f'{block}_{os.path.splitext(info[layer["file_key"]])[0]}_thumbnail.png'
            thumbnail_path = os.path.join(layer_dir, thumbnail_filename)

            if layer['type'] == 'raster':
                item = create_raster_item(location, block, info[layer['file_key']], file_path, layer_dir, thumbnail_path, style_path, base_dir)
            else:
                vector_desc_df = parse_vector_descriptions(style_path)
                item = create_vector_item(location, block, info[layer['file_key']], file_path, vector_desc_df, layer_dir, thumbnail_path, style_path, base_dir)
            
            block_catalog.add_item(item)
    
    block_catalog.set_self_href(os.path.join(block_dir, 'catalog.json'))
    block_catalog.normalize_and_save(block_dir, catalog_type=pystac.CatalogType.SELF_CONTAINED)
    print(f" STAC catalog created for block: {block} in {location}")

    location_catalog_path = os.path.join(location_dir, 'catalog.json')
    location_catalog_modified = False 

    if os.path.exists(location_catalog_path):
        location_catalog = pystac.read_file(location_catalog_path)
        print(f"Loaded existing location catalog: {location}")
    else:
        os.makedirs(location_dir, exist_ok=True)
        location_catalog = pystac.Catalog(
            id=location,
            title=f"STAC for {location}",
            description=f"STAC catalog for data in {location}"
        )
        location_catalog.set_self_href(location_catalog_path)
        print(f"Created new location catalog: {location}")
        location_catalog_modified = True

    child_id_to_add = block_catalog.id
    existing_child_ids = {child.id for child in location_catalog.get_children()} 
    
    if child_id_to_add not in existing_child_ids:
        child_to_add = pystac.read_file(os.path.join(block_dir, 'catalog.json'))
        location_catalog.add_child(child_to_add)
        location_catalog_modified = True 
        print(f"Added block '{block}' to location catalog '{location}'.")
    else:
        print(f"Block '{block}' already exists in location catalog '{location}'") 
    
    if location_catalog_modified:
        location_catalog.normalize_and_save(location_dir, catalog_type=pystac.CatalogType.SELF_CONTAINED)
        print(f"Updated location catalog for: {location}")

### 9. Generating the root catalog

In [98]:
def generate_root_catalog(blocks_info, base_dir, corestack_dir):
    root_catalog_path = os.path.join(corestack_dir, "catalog.json")

    if os.path.exists(root_catalog_path):
        root_catalog = pystac.read_file(root_catalog_path)
        print("Loaded existing root catalog.")
    else:
        root_catalog = pystac.Catalog(
            id="corestack",
            title="CorestackCatalogs",
            description="Root catalog containing all location-based sub-catalogs"
        )
        root_catalog.set_self_href(root_catalog_path) 
        print("Created new root catalog.")
    
    existing_root_children_ids = {child.id for child in root_catalog.get_children()}

    for info in blocks_info:
        location = info["location"]
        location_catalog_path = os.path.join(corestack_dir, location, "catalog.json")

        if os.path.exists(location_catalog_path):
            if location not in existing_root_children_ids:
                location_catalog = pystac.read_file(location_catalog_path)
                root_catalog.add_child(location_catalog)
                existing_root_children_ids.add(location)
                print(f"Added location catalog '{location}' to root catalog.")
            else:
                print(f"Location catalog '{location}' already linked in root catalog.")
        else:
            print(f"Warning: Location catalog not found for {location} at {location_catalog_path}")
                
    root_catalog.set_self_href(os.path.join(corestack_dir, "catalog.json"))
    root_catalog.normalize_and_save(corestack_dir, catalog_type=pystac.CatalogType.SELF_CONTAINED)
    print(f"Root catalog generated at {os.path.join(corestack_dir, 'catalog.json')}")

### 10. Executing the notebook

In [99]:
for block_info in blocks_info:
    print(f"Processing block: {block_info['block']}")
    generate_stac_for_block(block_info)
    
generate_root_catalog(blocks_info, base_dir="data/", corestack_dir="data/CorestackCatalogs")

Processing block: gobindpur
saraikela-kharsawan_gobindpur_2023-07-01_2024-06-30_LULCmap_10m.tif
2023-07-01 00:00:00
2024-06-30 00:00:00
Raster resolution (GSD): 9.940020840066722 meters
Unique values in raster data: [ 1.  2.  3.  4.  6.  7.  8.  9. 10. 11. 12. nan]
Parsed QML values: [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12]
Parsed QML colors: ['#ff0000', '#74ccf4', '#1ca3ec', '#0f5e9c', '#38761d', '#a9a9a9', '#bad93e', '#f59d22', '#ff9371', '#b3561d', '#a9a9a9']


/tmp/ipykernel_1626956/2983099978.py:13: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = mapping(gdf_wgs84.unary_union)


 STAC catalog created for block: gobindpur in jharkhand
Loaded existing location catalog: jharkhand
Block 'gobindpur' already exists in location catalog 'jharkhand'
Processing block: mirzapur
Mirzapur_Mirzapur_2023-07-01_2024-06-30_LULCmap_10m.tif
2023-07-01 00:00:00
2024-06-30 00:00:00
Raster resolution (GSD): 9.623295742250088 meters
Unique values in raster data: [ 1.  2.  3.  4.  6.  7.  8.  9. 10. 11. 12. nan]
Parsed QML values: [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12]
Parsed QML colors: ['#ff0000', '#74ccf4', '#1ca3ec', '#0f5e9c', '#38761d', '#a9a9a9', '#bad93e', '#f59d22', '#ff9371', '#b3561d', '#a9a9a9']


/tmp/ipykernel_1626956/2983099978.py:13: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = mapping(gdf_wgs84.unary_union)


 STAC catalog created for block: mirzapur in uttar_pradesh
Loaded existing location catalog: uttar_pradesh
Block 'mirzapur' already exists in location catalog 'uttar_pradesh'
Processing block: koraput
Narayanpatana_Koraput_2023-07-01_2024-06-30_LULCmap_10m.tif
2023-07-01 00:00:00
2024-06-30 00:00:00
Raster resolution (GSD): 9.827629984712452 meters
Unique values in raster data: [ 1.  2.  3.  4.  6.  7.  8.  9. 10. 11. 12. nan]
Parsed QML values: [1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12]
Parsed QML colors: ['#ff0000', '#74ccf4', '#1ca3ec', '#0f5e9c', '#38761d', '#a9a9a9', '#bad93e', '#f59d22', '#ff9371', '#b3561d', '#a9a9a9']


/tmp/ipykernel_1626956/2983099978.py:13: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = mapping(gdf_wgs84.unary_union)


 STAC catalog created for block: koraput in odisha
Loaded existing location catalog: odisha
Block 'koraput' already exists in location catalog 'odisha'
Processing block: Badlapur
jaunpur_badlapur_2019-07-01_2020-06-30_LULCmap_10m.tif
2019-07-01 00:00:00
2020-06-30 00:00:00
Raster resolution (GSD): 9.589087846259165 meters
Unique values in raster data: [ 1.  2.  3.  4.  6.  7.  8.  9. 10. 11. 12. nan]
Parsed QML values: [1, 2, 3, 4, 6, 7, 12, 8, 9, 10, 11]
Parsed QML colors: ['#ff0000', '#1ca3ec', '#1ca3ec', '#1ca3ec', '#73bb53', '#8ca991', '#eaa4f0', '#c6e46d', '#eee05d', '#f9b249', '#fb5139']


/tmp/ipykernel_1626956/2983099978.py:13: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = mapping(gdf_wgs84.unary_union)
/tmp/ipykernel_1626956/2983099978.py:13: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  geom = mapping(gdf_wgs84.unary_union)


 STAC catalog created for block: Badlapur in uttar_pradesh
Loaded existing location catalog: uttar_pradesh
Block 'Badlapur' already exists in location catalog 'uttar_pradesh'
Loaded existing root catalog.
Location catalog 'jharkhand' already linked in root catalog.
Location catalog 'uttar_pradesh' already linked in root catalog.
Location catalog 'odisha' already linked in root catalog.
Location catalog 'uttar_pradesh' already linked in root catalog.
Root catalog generated at data/CorestackCatalogs/catalog.json
